In [2]:
import pandas as pd

In [6]:
prints = pd.read_csv('./ukiyo-e_crawler/meta_data/ukiyo-e_artworks_details.csv').reset_index()
nouns = pd.read_csv('./text_processing/labels.csv')
nouns = nouns[['Unnamed: 0', 'LABELS']].rename(columns={'Unnamed: 0': 'index'})

C:\Users\marga\AppData\Local\Temp\ipykernel_21508\1436364997.py:2: DtypeWarning: Columns (4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191) have mixed types. Specify dtype option on import or set low_memory=False.
  nouns = pd.read_csv('./text_processing/labels.csv')


In [7]:
print(len(prints))
prints.head(1)

177985


,index,Image URL,Artist,Title,Date,Details,Source,Description,Similar Prints
0,0,https://data.ukiyo-e.org/bm/scaled/AN00418884_...,Okumura Masanobu,NaN,1711,https://www.britishmuseum.org/collection/objec...,http://www.britishmuseum.org/research/search_t...,Print. Sixth of series of twelve (first two mi...,[]


In [8]:
print(len(nouns))
nouns.head(1) # the lengths do not match...

177939


,index,LABELS
0,0,"['print', 'sixth', 'series', 'twelve', 'first'..."


In [9]:
prints_with_nouns = prints.merge(nouns, on='index') # this is problematic: 
# if i merge on linear index, some works won't be associated with the correct description... 
# i would need something else in labels.csv to merge on, such as the title
print(len(prints_with_nouns))

177939


In [3]:
test_full_data = pd.read_csv("./ukiyo-e_artworks_labels.csv")
print(len(test_full_data))
test_full_data.tail(3)

177985


,Unnamed: 0,Image URL,Artist,Title,Date,Details,Source,Description,Similar Prints,LABELS
177982,177982,https://data.ukiyo-e.org/artelino/scaled/31208...,Kusaka Kenji,Mt.Fuji is seen around Miya,1949.,http://www.artelino.com/archive/archivesearch_...,http://artelino.com/,"""Fuji Miya Fukin"". Mt. Fuji is seen from the a...","['https://ukiyo-e.org/image/artelino/19069g1',...","['see', 'area', 'near', 'see', 'around']"
177983,177983,https://data.ukiyo-e.org/artelino/scaled/50236...,Kusaka Kenji,Kiyomizu Temple,1948..,http://www.artelino.com/archive/archivesearch_...,http://artelino.com/,The performance platform of Kiyomizu temple in...,['https://ukiyo-e.org/image/jaodb/Kawai_Kenji-...,"['performance', 'platform', 'temple', 'autumn'..."
177984,177984,https://data.ukiyo-e.org/artelino/scaled/15906...,Kusaka Kenji,Ski Slope in Akino,Ca. 1950s.,http://www.artelino.com/archive/archivesearch_...,http://artelino.com/,Ski Slope in Akino.,['https://ukiyo-e.org/image/jaodb/Kawai_Kenji-...,"['ski', 'slope', 'ski', 'slope']"


In [ ]:
import ast

test_full_data['LABELS'] = test_full_data['LABELS'].apply(lambda x: ast.literal_eval(x))

In [20]:
import numpy as np

keywords = ['Emperor', 'Shogun', 'Government officials', 'Voting', 'Constitution',
              'Courtroom', 'Legal document', 'Contracts', 'Patent', 'Judge', 'Lawyer','Police', 'Prison',
              'School', 'School uniform', 'Textbooks', 'Scientific instrument',
              'Steamship', 'Telegraph poles', 'Brick house',
              'Factory', 'Steam-powered machinery', 'Worker', 'Railway', 'Train', 'Train station',
              'Soldier', 'Gun', 'Military uniform', 'Warship',
              'Kimono', 'Western style clothing', 'Gloves']

keywords = [x.lower() for x in keywords]

test = test_full_data.iloc[0]['LABELS']

def kw_in_labels(labels):
    res = {k: 0 for k in keywords}
    for l in labels:
        if (l in keywords):
            res[l] = res[l] + 1
    return res

test_full_data['kws'] = test_full_data['LABELS'].apply(lambda x: kw_in_labels(x))
# print(len(test_full_data[test_full_data['kws']==True]))
test_full_data[['LABELS', 'kws']].head()


full_res = {k: 0 for k in keywords}

# make the huge dictionary
for _, d in test_full_data.iterrows():
    dico = d.kws
    for k in keywords:
        full_res[k] = full_res[k] + dico[k]

print(full_res)

{'emperor': 699, 'shogun': 335, 'government officials': 0, 'voting': 0, 'constitution': 31, 'courtroom': 0, 'legal document': 0, 'contracts': 0, 'patent': 120, 'judge': 47, 'lawyer': 0, 'police': 40, 'prison': 9, 'school': 282, 'school uniform': 0, 'textbooks': 0, 'scientific instrument': 0, 'steamship': 16, 'telegraph poles': 0, 'brick house': 0, 'factory': 21, 'steam-powered machinery': 0, 'worker': 102, 'railway': 45, 'train': 85, 'train station': 0, 'soldier': 213, 'gun': 98, 'military uniform': 0, 'warship': 30, 'kimono': 1372, 'western style clothing': 0, 'gloves': 0}


In [22]:
keywords = ['Emperor', 'Shogun', 'Minister', 'Vote', 'Constitution',
              'Courtroom', 'Contract', 'Patent', 'Judge', 'Lawyer', 'Police', 'Prison',
              'School', 'Uniform', 'Textbook', 'Scientific instrument',
              'Steamship', 'Telegraph', 'Brick',
              'Factory', 'Steam', 'Worker', 'Railway', 'Train',
              'Soldier', 'Gun', 'Warship',
              'Kimono', 'Suit', 'Gown', 'Glove']

keywords = [x.lower() for x in keywords]

test_full_data['kws2'] = test_full_data['LABELS'].apply(lambda x: kw_in_labels(x))
test_full_data[['LABELS', 'kws2']].head()

full_res = {k: 0 for k in keywords}

# make the huge dictionary
for _, d in test_full_data.iterrows():
    dico = d.kws2
    for k in keywords:
        full_res[k] = full_res[k] + dico[k]

print(full_res)

{'emperor': 699, 'shogun': 335, 'minister': 65, 'vote': 0, 'constitution': 31, 'courtroom': 0, 'contract': 17, 'patent': 120, 'judge': 47, 'lawyer': 0, 'police': 40, 'prison': 9, 'school': 282, 'uniform': 33, 'textbook': 4, 'scientific instrument': 0, 'steamship': 16, 'telegraph': 9, 'brick': 24, 'factory': 21, 'steam': 73, 'worker': 102, 'railway': 45, 'train': 85, 'soldier': 213, 'gun': 98, 'warship': 30, 'kimono': 1372, 'suit': 42, 'gown': 28, 'glove': 12}


In [42]:
dico_per_image = test_full_data[['kws2']]
type(dico_per_image)

pandas.core.frame.DataFrame

In [51]:
word_matrix = pd.DataFrame(index=np.arange(len(dico_per_image)), columns=full_res.keys())

for i, d in dico_per_image.iterrows():
    for k in d['kws2'].keys():
        word_matrix[k].loc[i] = d['kws2'][k]

word_matrix

KeyboardInterrupt: 

After this, two things to do:

- Calculate similarity (find package, and get similarity from the word_matrix I'm creating)
- Merge the full_test_data with the time dataframe, and for each keyword in the kws, create a graph showing time evolution of the word's appearance.